In [ ]:
!pip install datasets
!pip install transformers accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch
from google.colab import drive
import json
drive.mount('/content/drive')
# Step 1: Load a pre-trained model and tokenizer
model_name = "gpt2"  # You can use "bert-base-uncased", "gpt-neo", etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token
model = AutoModelForCausalLM.from_pretrained(model_name)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Step 2: Load your custom dataset from a JSON file
# Upload your JSON file to Colab
json_file_path = '/content/drive/My Drive/JSON/scraped_paragraphs.json'

# Load the JSON file
with open(json_file_path, "r") as f:
    json_data = json.load(f)

# Extract the "Paragraphs" field from the JSON data
custom_data_json = [item["Paragraphs"] for item in json_data]

# Load TXT dataset
txt_file_path = '/content/drive/My Drive/TXT/merged_output.txt'
with open(txt_file_path, "r") as f:
    custom_data_txt = [line.strip() for line in f.readlines() if line.strip()]

# Combine both datasets
custom_data = custom_data_json + custom_data_txt

# Convert the dataset into a Hugging Face Dataset object
dataset = Dataset.from_dict({"text": custom_data})

# Tokenize the dataset
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Use input_ids as labels
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split the dataset into training and validation sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

Map:   0%|          | 0/79395 [00:00<?, ? examples/s]

In [ ]:
# Step 3: Fine-tune the model
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/content/fine_tuned_model",  # Save to a temporary folder in Colab
    overwrite_output_dir=True,
    num_train_epochs=5,  # Increase the number of epochs
    per_device_train_batch_size=4,  # Increase the batch size
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",  # Save model after each epoch
    logging_dir="./logs",  # Directory for logs
    logging_steps=10,
    save_total_limit=2,  # Limit the number of saved checkpoints
    fp16=True,  # Use mixed precision for faster training (if supported)
    learning_rate=7e-5,  # Adjust the learning rate
    weight_decay=0.01,  # Add weight decay for regularization
    push_to_hub=False,  # Set to True if you want to upload to Hugging Face Hub
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
)

# Start fine-tuning
trainer.train()
# Step 4: Save the fine-tuned model
trainer.save_model("/content/fine_tuned_model")
tokenizer.save_pretrained("/content/fine_tuned_model")

print("Fine-tuning complete! Model saved to '/content/fine_tuned_model'.")

# Step 5: Compress the model folder
!tar -czvf fine_tuned_model.tar.gz -C /content/fine_tuned_model .

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abhaywadsinge (abhaywadsinge-bosch-global) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss


In [ ]:
# Step 6: Download the compressed file to your local machine
from google.colab import files
files.download("fine_tuned_model.tar.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

# Use the model for inference
input_text = "who was harry potter?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


who was harry potter? I don't think so. I think he was a bit of a dick. I think he was a bit of a dick to me. I think he was a bit of a jerk. I think he was a bit of a jerk to me. I think he was a bit of a jerk to me. I think he was a bit of a jerk to me. I think he was a bit of a jerk to me. I think he was a bit of
